In [1]:
from selenium import webdriver
import time
import random
import pandas as pd
import xlwings as xw
import twder
from tqdm import tqdm

In [2]:
# 爬下所有基金的網址
option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome(options=option)
driver.maximize_window()

urls = []

for page in range(1,6):
    # 全球市場科技類
    url = f'https://fund.cnyes.com/search/?categoryAbbr=C46&investmentArea=A3&order=inceptionDate&page={page}&sort=asc'
    driver.get(url)
    driver.implicitly_wait(8)
    
    for i in range(1,23):  # 每頁23列
        try:
            href = driver.find_element_by_css_selector(f'#target_tbody > tr:nth-child({i}) > td._1OU0Y._2rwNH > a').get_attribute('href')
            urls.append(href.split('/report/')[0])
            
        except:
            continue
            
    time.sleep(1+random.random())
    
        
driver.close()

# 存入csv
urls = pd.DataFrame(urls)
urls.to_csv('fund url.csv', header=0, index=0, mode='w')

In [3]:
# 逐網址爬資料
df = pd.read_csv('fund url.csv', header=None)

option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome(options=option)
driver.maximize_window()

wb = xw.Book()
sheet = wb.sheets['工作表1']

# Excel欄位名稱
sheet.cells(1, "A").value = '基金名稱'
sheet.cells(1, "B").value = '1年基金績效'
sheet.cells(1, "C").value = '3年基金績效'
sheet.cells(1, "D").value = '5年基金績效'
sheet.cells(1, "E").value = '10年基金績效'
sheet.cells(1, "F").value = '平均績效'
sheet.cells(1, "G").value = '基金規模'
sheet.cells(1, "H").value = '基金規模(更新日期)'
sheet.cells(1, "I").value = '基金註冊地'
sheet.cells(1, "J").value = '投資地區'
sheet.cells(1, "K").value = '計價幣別'
sheet.cells(1, "L").value = '基金組別'
sheet.cells(1, "M").value = '成立日期'
sheet.cells(1, "N").value = '風險評等'
sheet.cells(1, "O").value = '晨星評等'
sheet.cells(1, "P").value = '最低認購金額'
sheet.cells(1, "Q").value = '申購手續費'
sheet.cells(1, "R").value = '管理費'
sheet.cells(1, "S").value = '最高管理費'
sheet.cells(1, "T").value = '遞延費'
sheet.cells(1, "U").value = '最高遞延費'
sheet.cells(1, "V").value = '分銷費'
sheet.cells(1, "W").value = '保管費'
sheet.cells(1, "X").value = '資產配置(現金)'
sheet.cells(1, "Y").value = '資產配置(股票)'
sheet.cells(1, "Z").value = '資產配置(債券)'
sheet.cells(1, "AA").value = '資產配置(其他)'
sheet.cells(1, "AB").value = '1年夏普值'
sheet.cells(1, "AC").value = '3年夏普值'
sheet.cells(1, "AD").value = '5年夏普值'

# 即期賣出報價
USD = float(twder.now('USD')[4]) #美金即期賣出
HKD = float(twder.now('HKD')[4]) #港幣即期賣出
GBP = float(twder.now('GBP')[4]) #英鎊即期賣出
AUD = float(twder.now('AUD')[4]) #澳幣即期賣出
CAD = float(twder.now('CAD')[4]) #加拿大幣即期賣出
SGD = float(twder.now('SGD')[4]) #新加坡幣即期賣出
CHF = float(twder.now('CHF')[4]) #瑞士法郎即期賣出
JPY = float(twder.now('JPY')[4]) #日圓即期賣出
ZAR = float(twder.now('ZAR')[4]) #南非幣即期賣出
SEK = float(twder.now('SEK')[4]) #瑞典幣即期賣出
NZD = float(twder.now('NZD')[4]) #紐元即期賣出
THB = float(twder.now('THB')[4]) #泰幣即期賣出
#PHP = float(twder.now('PHP')[4]) #菲國比索即期賣出
#IDR = float(twder.now('IDR')[4]) #印尼幣即期賣出
EUR = float(twder.now('EUR')[4]) #歐元即期賣出
#KRW = float(twder.now('KRW')[4]) #韓元即期賣出
#VND = float(twder.now('VND')[4]) #越南盾即期賣出
#MYR = float(twder.now('MYR')[4]) #馬來幣即期賣出
CNY = float(twder.now('CNY')[4]) #人民幣即期賣出

# 轉美金
HKDtoUSD = HKD / USD
GBPtoUSD = GBP / USD
AUDtoUSD = AUD / USD
CADtoUSD = CAD / USD
SGDtoUSD = SGD / USD
CHFtoUSD = CHF / USD
JPYtoUSD = JPY / USD
ZARtoUSD = ZAR / USD
SEKtoUSD = SEK / USD
NZDtoUSD = NZD / USD
THBtoUSD = THB / USD
#PHPtoUSD = PHP / USD
#IDRtoUSD = IDR / USD
EURtoUSD = EUR / USD
#KRWtoUSD = KRW / USD
#VNDtoUSD = VND / USD
#MYRtoUSD = MYR / USD
CNYtoUSD = CNY / USD

count = 0

for i in tqdm(range(len(df))):
    try:
        count += 1
        url = df.loc[i,0]

        ### 基金概況
        driver.get(url)
        ## 績效表現
        # 1年基金績效
        one_year_performance = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > div:nth-child(3) > section:nth-child(1) > table > tbody > tr:nth-child(1) > td:nth-child(6)').text
        sheet.cells(count+1, "B").value = one_year_performance

        # 3年基金績效
        three_year_performance = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > div:nth-child(3) > section:nth-child(1) > table > tbody > tr:nth-child(1) > td:nth-child(7)').text
        sheet.cells(count+1, "C").value = three_year_performance

        # 5年基金績效
        five_year_performance = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > div:nth-child(3) > section:nth-child(1) > table > tbody > tr:nth-child(1) > td:nth-child(8)').text
        sheet.cells(count+1, "D").value = five_year_performance

        # 10年基金績效
        ten_year_performance = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > div:nth-child(3) > section:nth-child(1) > table > tbody > tr:nth-child(1) > td:nth-child(9)').text
        sheet.cells(count+1, "E").value = ten_year_performance

        time.sleep(1+random.random())

        ### 基金檔案
        driver.get(url + '/document')
        ## 基本資料
        # 基金名稱            
        fund_id = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(1) > td').text
        sheet.cells(count+1, "A").value = fund_id

        # 基金規模
        # 抓取基金規模並換算成以美金為單位
        fund_currency_str = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(5) > td > span')
        fund_size_currency = fund_currency_str.text.split()[0]
        fund_size = int(fund_currency_str.text.split()[1].replace(',',''))

        if fund_size_currency == 'USD':
            fund_size = fund_size
        elif fund_size_currency == 'HKD':
            fund_size = fund_size * HKDtoUSD
        elif fund_size_currency == 'GBP':
            fund_size = fund_size * GBPtoUSD
        elif fund_size_currency == 'AUD':
            fund_size = fund_size * AUDtoUSD
        elif fund_size_currency == 'CAD':
            fund_size = fund_size * CADtoUSD
        elif fund_size_currency == 'SGD':
            fund_size = fund_size * SGDtoUSD
        elif fund_size_currency == 'CHF':
            fund_size = fund_size * CHFtoUSD
        elif fund_size_currency == 'JPY':
            fund_size = fund_size * JPYtoUSD
        elif fund_size_currency == 'ZAR':
            fund_size = fund_size * ZARtoUSD
        elif fund_size_currency == 'SEK':
            fund_size = fund_size * SEKtoUSD
        elif fund_size_currency == 'NZD':
            fund_size = fund_size * NZDtoUSD
        elif fund_size_currency == 'THB':
            fund_size = fund_size * THBtoUSD
        #elif fund_size_currency == 'PHP':
        #    fund_size = fund_size * PHPtoUSD
        #elif fund_size_currency == 'IDR':
        #    fund_size = fund_size * IDRtoUSD
        elif fund_size_currency == 'EUR':
            fund_size = fund_size * EURtoUSD
        #elif fund_size_currency == 'KRW':
        #    fund_size = fund_size * KRWtoUSD
        #elif fund_size_currency == 'VND':
        #    fund_size = fund_size * VNDtoUSD
        #elif fund_size_currency == 'MYR':
        #    fund_size = fund_size * MYRtoUSD
        elif fund_size_currency == 'CNY':
            fund_size = fund_size * CNYtoUSD
        elif fund_size_currency == 'TWD':
            fund_size = fund_size / USD

        sheet.cells(count+1, "G").value = fund_size

        # 基金規模(更新日期)
        fund_currency_update = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(5) > td > span > span:nth-child(2)').text
        sheet.cells(count+1, "H").value = fund_currency_update

        # 基金註冊地
        fund_registration = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(6) > td:nth-child(2)').text
        sheet.cells(count+1, "I").value = fund_registration

        # 投資地區
        investment_area = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(6) > td:nth-child(4)').text
        sheet.cells(count+1, "J").value = investment_area

        # 計價幣別
        currency = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(7) > td:nth-child(2)').text
        sheet.cells(count+1, "K").value = currency

        # 基金組別
        fund_group = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(7) > td:nth-child(4)').text
        sheet.cells(count+1, "L").value = fund_group

        # 成立日期
        establish_date = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(9) > td:nth-child(4) > span').text
        sheet.cells(count+1, "M").value = establish_date

        # 風險評等
        risk_rating = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(2) > tbody > tr:nth-child(10) > td:nth-child(2)').text
        sheet.cells(count+1, "N").value = risk_rating

        # 晨星評等
        star_rating = 0
        stars = driver.find_elements_by_css_selector('img')
        for i in range(len(stars)):
            if stars[i].get_attribute('src') == 'https://fund.cnyes.com/dist/ee3ff1103ca031b33948607d0e2db079.svg':
                star_rating += 1
        sheet.cells(count+1, "O").value = star_rating

        ## 基金費用
        # 最低認購金額
        min_subscription = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(4) > tbody > tr:nth-child(1) > td:nth-child(2)').text
        sheet.cells(count+1, "P").value = min_subscription

        # 申購手續費
        handling_fee = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(4) > tbody > tr:nth-child(1) > td:nth-child(4)').text
        sheet.cells(count+1, "Q").value = handling_fee

        # 管理費
        management_fee = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(4) > tbody > tr:nth-child(2) > td:nth-child(2)').text
        sheet.cells(count+1, "R").value = management_fee

        # 最高管理費
        max_management_fee = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(4) > tbody > tr:nth-child(2) > td:nth-child(4)').text
        sheet.cells(count+1, "S").value = max_management_fee

        # 遞延費
        deferred_fee = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(4) > tbody > tr:nth-child(3) > td:nth-child(2)').text
        sheet.cells(count+1, "T").value = deferred_fee

        # 最高遞延費
        max_deferred_fee = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(4) > tbody > tr:nth-child(3) > td:nth-child(4)').text
        sheet.cells(count+1, "U").value = max_deferred_fee

        # 分銷費
        distribution_fee = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(4) > tbody > tr:nth-child(4) > td:nth-child(2)').text
        sheet.cells(count+1, "V").value = distribution_fee

        # 保管費
        custody_fee = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > table:nth-child(4) > tbody > tr:nth-child(4) > td:nth-child(4)').text
        sheet.cells(count+1, "W").value = custody_fee

        time.sleep(1+random.random())

        ### 持股明細
        driver.get(url + '/shareholding')
        ## 資產配置
        # 現金
        cash = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > section._20O20 > div:nth-child(1) > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > span._2n33c._2lIZk').text
        sheet.cells(count+1, "X").value = cash

        # 股票
        stock = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > section._20O20 > div:nth-child(1) > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > span._2n33c._2o4G9').text
        sheet.cells(count+1, "Y").value = stock

        # 債券
        bond = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > section._20O20 > div:nth-child(1) > div > table > tbody > tr:nth-child(2) > td:nth-child(1) > span._2n33c._3T2bs').text
        sheet.cells(count+1, "Z").value = bond

        # 其他
        other = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > section._20O20 > div:nth-child(1) > div > table > tbody > tr:nth-child(2) > td:nth-child(2) > span._2n33c._3h1pe').text
        sheet.cells(count+1, "AA").value = other

        time.sleep(1+random.random())

        ### 風險評等
        driver.get(url + '/risk')
        ## 風險資料
        # 1年夏普值
        one_year_sharpe = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > div > div._3_j1t > table > tbody > tr:nth-child(1) > td:nth-child(2)').text
        sheet.cells(count+1, "AB").value = one_year_sharpe

        # 3年夏普值
        three_year_sharpe = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > div > div._3_j1t > table > tbody > tr:nth-child(1) > td:nth-child(3)').text
        sheet.cells(count+1, "AC").value = three_year_sharpe

        # 5年夏普值
        five_year_sharpe = driver.find_element_by_css_selector('#anue-ga-wrapper > div > div._2hZZZ > div:nth-child(2) > main.main > section > main > div > div._3_j1t > table > tbody > tr:nth-child(1) > td:nth-child(4)').text
        sheet.cells(count+1, "AD").value = five_year_sharpe

        time.sleep(1+random.random())

    except:
        count -= 1
        continue

driver.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [16:23<00:00, 10.14s/it]
